In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests

In [3]:
wifi = pd.read_parquet('./wifi_data_2days.parquet')

In [7]:
wifi.shape

(9926508, 11)

In [8]:
wifi.columns

Index(['time', 'MAC', 'IP', 'APM', 'APN', 'building_id', 'zone', 'section',
       'floor', 'SSID', 'Username'],
      dtype='object')

In [4]:
wifi.sample(3)

,time,MAC,IP,APM,APN,building_id,zone,section,floor,SSID,Username
2108650,2025-04-14 17:52:07,cd589283d0d3316baacbdebc5d78074553a71ae780fdef...,None,None,095-148-o,095,148,o,1o,eduroam,7fdeea22698cf4a7d52e2fcc79decb916d72b6a24f0411...
5587586,2025-04-15 13:57:11,b40f88e89ffafc6ed1b42b20c97d83d417be78e5da8c10...,None,None,153-Outdoor-West,153,Outdoor,West,O,eduroam,71b127fd5e7fee8b25c1313782ce1f0e3887a7057f77cf...
1264297,2025-04-14 14:45:13,dd23260b8a3b8c10c99a97bf9090e2f313d7c1b27cca65...,None,None,081-S106,081,S106,None,S,eduroam,c597ee7b8802a24fa29565fd180e3c5f67a285a3ace35f...


### Columns

- MAC is the device MAC address
- IP is the device IP address
- APM is the access point MAC address I am pretty sure
- APN is access point name, you can think about this as the specific wifi router
- building_id / zone / section are all derivatives of the APN
- floor is which floor the apn is on, duh! 
- SSID is the name of the wifi network

#### I'll help you fetch geo-data for campus building outlines and attach them to the wifi data for you

In [13]:
url = "https://services5.arcgis.com/7WaXTZEsI88qiQGw/arcgis/rest/services/Utility_Services_View_Only/FeatureServer/0/query?where=1=1&f=geojson&outFields=*"

print("Fetching GeoJSON data...")
response = requests.get(url)
if response.status_code == 200:
    geojson_data = response.json()
    print(f"Successfully fetched data with {len(geojson_data['features'])} features")
else:
    print(f"Failed to fetch data: {response.status_code}")

campus = gpd.GeoDataFrame.from_features(geojson_data['features'], crs="EPSG:4326")

Fetching GeoJSON data...
Successfully fetched data with 246 features


In [15]:
# remove the letters from the building_id for consistency (edit this if you want)
wifi['bid_no_letters'] = wifi['building_id'].str.extract(r'(\d+)')

In [16]:
merged = wifi.merge(campus[['geometry', 'buildings_BLDG_CODE']], left_on='bid_no_letters', right_on='buildings_BLDG_CODE', how='left')

merged_gdf = gpd.GeoDataFrame(merged, geometry='geometry', crs="EPSG:4326")

In [17]:
merged_gdf.sample(3)

,time,MAC,IP,APM,APN,building_id,zone,section,floor,SSID,Username,bid_no_letters,geometry,buildings_BLDG_CODE
2163431,2025-04-14 12:51:02,5203f079af8683ee3c1a788381ae0487f50eda5187e602...,None,84:3d:c6:6e:71:c0,094-Laundry,094,Laundry,None,L,eduroam,1602ec4b0bdb074637f5c0594574754c74d3d6c3395213...,094,"POLYGON ((-84.39107 33.77404, -84.39107 33.774...",094
6863715,2025-04-14 19:27:29,0ca03bfe099964b2e857ef5cb13e2a0eb99aa04104aec8...,None,None,167-2284,167,2284,None,2,eduroam,dc70cfd3269d23f86748db1890879d04899a625ae8f2a2...,167,"MULTIPOLYGON (((-84.39639 33.77955, -84.39637 ...",167
7563076,2025-04-14 21:11:34,b4231ba1b92eae1f1752574a5cd0377e513c6f6c629bd3...,128.61.56.197,None,172-117A-o,172,117A,o,1o,eduroam,4815fa983d1fac879dd09de790adb7b479770f9617ba9e...,172,"MULTIPOLYGON (((-84.3883 33.77617, -84.38832 3...",172


In [ ]:
# Create a custom map with no shadows/blotches
import folium
from folium import plugins

# Create base map
m = folium.Map(location=[merged_gdf.geometry.centroid.y.mean(), merged_gdf.geometry.centroid.x.mean()], 
               zoom_start=15, tiles='OpenStreetMap')

# Add buildings with custom styling to remove shadows
sample_data = merged_gdf.sample(100)

# Define custom style function to remove shadows
def style_function(feature):
    return {
        'fillColor': 'lightblue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.7,
        'opacity': 1,
        'shadow': False,  # Remove shadows
        'shadowColor': 'transparent',  # Make shadow transparent
        'shadowOpacity': 0,  # No shadow opacity
        'shadowBlur': 0,  # No shadow blur
        'shadowOffsetX': 0,  # No shadow offset
        'shadowOffsetY': 0,  # No shadow offset
    }

# Add GeoJSON layer with custom styling
folium.GeoJson(
    sample_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['building_id', 'APN'], aliases=['Building ID', 'Access Point'])
).add_to(m)

# Display the map
m

In [20]:
# change the time to a datetime object
merged_gdf['time'] = pd.to_datetime(merged_gdf['time'])
merged_gdf['hour'] = merged_gdf['time'].dt.hour

In [ ]:
# Alternative approach: Use explore() with custom styling to remove shadows
# This method removes the black blotches by customizing the map styling
sample_data = merged_gdf.sample(100)

# Create map with custom styling to remove shadows
m = sample_data.explore(
    style_kwds={
        'fillColor': 'lightblue',
        'color': 'blue', 
        'weight': 2,
        'fillOpacity': 0.7,
        'opacity': 1,
        'shadow': False,  # Explicitly disable shadows
        'shadowColor': 'transparent',
        'shadowOpacity': 0,
        'shadowBlur': 0,
        'shadowOffsetX': 0,
        'shadowOffsetY': 0,
    },
    tooltip=True,
    popup=True,
    tiles='OpenStreetMap'
)

# Additional CSS to ensure no shadows are rendered
m.get_root().html.add_child(folium.Element("""
<style>
    .leaflet-shadow-pane { display: none !important; }
    .leaflet-marker-shadow { display: none !important; }
    .leaflet-shadow { display: none !important; }
    [class*="shadow"] { display: none !important; }
</style>
"""))

m


In [ ]:
# Method 3: Complete shadow removal using JavaScript injection
# This is the most comprehensive approach to eliminate all shadow effects

sample_data = merged_gdf.sample(100)

# Create the map
m = sample_data.explore(
    style_kwds={
        'fillColor': 'lightblue',
        'color': 'blue',
        'weight': 2, 
        'fillOpacity': 0.7,
        'opacity': 1
    },
    tooltip=True,
    popup=True,
    tiles='OpenStreetMap'
)

# Inject JavaScript to completely disable shadows
m.get_root().html.add_child(folium.Element("""
<script>
// Disable all shadow rendering in Leaflet
document.addEventListener('DOMContentLoaded', function() {
    // Override shadow rendering methods
    if (window.L && window.L.Layer) {
        const originalAddTo = window.L.Layer.prototype.addTo;
        window.L.Layer.prototype.addTo = function(map) {
            const result = originalAddTo.call(this, map);
            // Remove any shadow elements
            if (this._shadow) {
                this._shadow.remove();
                this._shadow = null;
            }
            return result;
        };
    }
    
    // Remove existing shadows
    setTimeout(function() {
        const shadows = document.querySelectorAll('.leaflet-shadow-pane, .leaflet-marker-shadow, [class*="shadow"]');
        shadows.forEach(function(shadow) {
            shadow.style.display = 'none';
            shadow.remove();
        });
    }, 100);
});
</script>

<style>
/* Comprehensive shadow removal */
.leaflet-shadow-pane,
.leaflet-marker-shadow, 
.leaflet-shadow,
[class*="shadow"],
.leaflet-interactive[style*="shadow"],
svg[style*="shadow"] {
    display: none !important;
    visibility: hidden !important;
    opacity: 0 !important;
}

/* Remove any pseudo-element shadows */
.leaflet-interactive::before,
.leaflet-interactive::after {
    display: none !important;
}
</style>
"""))

m
